In [6]:
import requests
from bs4 import BeautifulSoup
import bs4
import re
import pandas as pd
from datetime import datetime

### Get data from forum

[Forum](https://www.mentalhealthforum.net)

In [7]:
def _mentalHealthForumScraper(threadnum=97215, pagenum=1):
    url = 'https://www.mentalhealthforum.net/forum/thread{}-{}.html'.format(threadnum, pagenum)
    forum_html = requests.get(url)
    forum_json = forum_html.text

    forum_soup = BeautifulSoup(forum_json, 'html.parser')
  
  
    users_posts = []
    # get each post
    for post in forum_soup.find_all("li", "postbitlegacy postbitim postcontainer old"):
        # find the username of the poster
        post_user = post.find("div", {"class":"username_container"})

        m = re.search("(?<=(\n))[^\s]+", post_user.text)
        username = m.group(0)
        post_html = post.find("blockquote", {"class":"postcontent restore "})

        # find date of post
        time_str = post.find("span", "date").text
        time_str = re.sub(u'\xa0', u' ', time_str)
        time_obj = datetime.strptime(time_str, "%d-%m-%y, %H:%M")

        # quoted someone else
        quote_authors_html = post_html.find_all("div", {"class":"bbcode_postedby"})
        quote_author = ""
        if len(quote_authors_html) > 0:
            for quote_author_html in quote_authors_html:
                quote_author += quote_author_html.find("strong").text + ', '
            quote_author = quote_author[:-2]

        # post.contents will give me the "contents" of the post, which means
        # everything without the <div> tag can be thrown away, and everything
        # with the link we can keep the text for

        # first 2 elements are the '\n' and the <div></div> tag
        post_text = ""
        for content in post_html.contents:
            if type(content) is bs4.element.NavigableString:
                # if the content is text, add to overall text
                post_text += content
            elif content.find("div") is None:
                # ignoring quotes (div tags), find text surrounded by html (emoji, break tags, links, italics, etc.)
                if not re.match("<br/>", str(content)):
                    try:
                        post_text += '(' + content['title'] + ')'
                    except KeyError as ex:
                        if content.text is not None:
                            post_text += " " + content.text + " "
                        else:
                            print('uncategorizable text: ' + content)

        # edit out any '\n' s
        post_final = re.sub("\\n", "", post_text)

        users_posts.append([time_obj, username, post_final, quote_author])

    return users_posts

In [8]:
def getNPosts(threadnum=97215, n=50):
    posts = []
    pagenum = 1
    while len(posts) < n:
        newPosts = _mentalHealthForumScraper(threadnum, pagenum)
        for post in newPosts:
            posts.append(post)
        pagenum += 1
    return pd.DataFrame(posts, columns=['date', 'username', 'post content', 'quoted user'])

In [ ]:
raw_data_directory = '../../data/raw/posts/'
forum_ids = [97215, 186460, 30827, 41312]
output_files = ['singlesentence', 'mamistruggling', 'battlingdepressionnomeds', 'lonely']
num_posts = 200

for forum_id, output_file in zip(forum_ids, output_files):
    getNPosts(forum_id, num_posts).to_csv(raw_data_directory + output_file + str(num_posts) + '.csv', index=False)

In [10]:
getNPosts(41312, 20)

,date,username,post content,quoted user
0,2012-05-25 22:01:00,bulbie,...all of the time. It's crippling me. (Nod 2),
1,2012-05-25 22:08:00,manekineko,"Hi bulbie, I share your pain! That's why I lik...",
2,2012-05-25 22:10:00,Starless,(Group Hug),
3,2012-05-25 22:40:00,bulbie,"I'm completely isolated here, that really does...",
4,2012-05-25 22:46:00,Unloved,hiya Bulbie.. You are NOT alone in your feelin...,
5,2012-05-25 22:51:00,manekineko,You can talk to your friends here (Smile) AWWW...,
6,2012-05-25 22:55:00,Allyke,Love you Bulbie (Group Hug) xx,
7,2012-05-25 23:25:00,Sugarplum,You sound fantastic to me. I'd go out with you...,Allyke
8,2012-05-26 11:24:00,bulbie,Thank you so much everyone.I'm hoping a visit ...,
9,2012-05-26 11:52:00,Unloved,"morning Bulbie, its nice to hear your going ou...",
